# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [20]:


#Instalando o tweepy
#!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [21]:
import tweepy
import math
import os.path
import os
import pandas as pd
import json
from random import shuffle
from functools import reduce
from operator import mul

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [22]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [23]:
#Produto escolhido:
produto = 'fifa 17'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [24]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [25]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [26]:
xl=pd.ExcelFile("fifa 17.xlsx")
xl.sheet_names
[u'Treinamento', u'Teste']

['Treinamento', 'Teste']

In [27]:
df = xl.parse("Treinamento")

df['Treinamento'] = df['Treinamento'].str.replace(",","")
df['Treinamento'] = df['Treinamento'].str.replace(".","")
df['Treinamento'] = df['Treinamento'].str.replace("'","")
df['Treinamento'] = df['Treinamento'].str.replace('"',"")
df['Treinamento'] = df['Treinamento'].str.replace("  "," ")
df['Treinamento'] = df['Treinamento'].str.replace(" a "," ")
df['Treinamento'] = df['Treinamento'].str.replace(" as "," ")
df['Treinamento'] = df['Treinamento'].str.replace(" o "," ")
df['Treinamento'] = df['Treinamento'].str.replace(" os "," ")
df['Treinamento'] = df['Treinamento'].str.replace(")","")
df['Treinamento'] = df['Treinamento'].str.replace("(","")
df['Treinamento'] = df['Treinamento'].str.replace("-","")
df['Treinamento'] = df['Treinamento'].str.replace("!","")
df['Treinamento'] = df['Treinamento'].str.replace("?","")

df['Treinamento'] = df['Treinamento'].str.lower()

In [28]:
df2 = xl.parse("Teste")

df2['Teste'] = df2['Teste'].str.replace(",","")
df2['Teste'] = df2['Teste'].str.replace(".","")
df2['Teste'] = df2['Teste'].str.replace("'","")
df2['Teste'] = df2['Teste'].str.replace('"',"")
df2['Teste'] = df2['Teste'].str.replace("  "," ")
df2['Teste'] = df2['Teste'].str.replace(" a "," ")
df2['Teste'] = df2['Teste'].str.replace(" as "," ")
df2['Teste'] = df2['Teste'].str.replace(" o "," ")
df2['Teste'] = df2['Teste'].str.replace(" os "," ")
df2['Teste'] = df2['Teste'].str.replace(")","")
df2['Teste'] = df2['Teste'].str.replace("(","")
df2['Teste'] = df2['Teste'].str.replace("-","")
df2['Teste'] = df2['Teste'].str.replace("!","")
df2['Teste'] = df2['Teste'].str.replace("?","")

df2['Teste'] = df2['Teste'].str.lower()

In [38]:
def repetido(a):
    return list(set(a))



relevante = []
relevante_srep = []
n_relevante = []
n_relevante_srep = []
todas = []
todas_srep = []

for i in range(len(df)): 
    todas+=(list(df["Treinamento"][i].split()))
    

for i in range(len(df)):
    if df["B1"][i]==1:
        relevante+=(list(df["Treinamento"][i].split()))
        
    if df["B1"][i]==2:
        n_relevante+=(list(df["Treinamento"][i].split()))
        
        

relevante_srep = repetido(relevante)
n_relevante_srep = repetido(n_relevante)
todas_srep = repetido(todas)


numero_relevante = len(relevante)
numero_relevante_srep = len(relevante_srep)
numero_n_relevante= len(n_relevante)
numero_n_relevante_srep= len(n_relevante_srep)
numero_todas = len(todas)
numero_todas_srep = len(todas_srep)

In [39]:
palavra_relevante={}
pr=[]

i=0
while i<len(relevante):
    if relevante[i] in palavra_relevante:
        palavra_relevante[relevante[i]]=palavra_relevante[relevante[i]]+1
    else:
        palavra_relevante[relevante[i]]=1
        pr.append(relevante[i])
        
    i+=1

In [40]:

palavra_n_relevante={}
pnr=[]

i=0
while i<len(n_relevante):
    if n_relevante[i] in palavra_n_relevante:
        palavra_n_relevante[n_relevante[i]]=palavra_n_relevante[n_relevante[i]]+1
    else:
        palavra_n_relevante[n_relevante[i]]=1
        pnr.append(n_relevante[i])
    i+=1

In [47]:
palavra_prob_relevante={}
palavra_prob_nrelevante={}

i=0
while i<len(relevante):
    palavra_prob_relevante[relevante[i]]=((palavra_relevante[relevante[i]]+1)/(numero_relevante+numero_todas_srep))
    i+=1
    
while i<len(n_relevante):
    palavra_prob_nrelevante[n_relevante[i]]=((palavra_n_relevante[n_relevante[i]]+1)/(numero_n_relevante+numero_todas_srep))
    i+=1

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [48]:
teste=[]
probabilidade_relevante_t=[]
probabilidade_n_relevante_t=[]


for i in df2["Teste"]:
    a=i.split()
    teste.append(a)
    
    
    
for t in teste:
    p=[1]
    for e in t:
        if e in palavra_prob_relevante:
            p.append(palavra_prob_relevante[e])
        else:
            v = 1 / (numero_relevante + numero_todas_srep)
            p.append(v)
            
    a = reduce(mul,p)
    probabilidade_relevante_t.append(a)
    
    
    
for t in teste:
    p=[1]
    for e in t:
        if e in palavra_prob_nrelevante:
            p.append(palavra_prob_nrelevante[e])
        else:
            v = 1 / (numero_n_relevante + numero_todas_srep)
            p.append(v)
            
    a = reduce(mul,p)
    probabilidade_n_relevante_t.append(a)

In [49]:
listaFinalRelevancia = []
i=0
while i <(len(teste)):
    if probabilidade_relevante_t[i] > probabilidade_n_relevante_t[i]:
        a=1
        listaFinalRelevancia.append(a)
        
        
    elif probabilidade_relevante_t[i] < probabilidade_n_relevante_t[i]:
        a=2
        listaFinalRelevancia.append(a)
        
       
    elif probabilidade_relevante_t[i] == probabilidade_n_relevante_t[i]:
        a=0
        listaFinalRelevancia.append(a)
        
    i+=1
    

    
df2["B1"] = pd.Series(listaFinalRelevancia)

In [53]:
resultados=[]

for i in range(len(listaFinalRelevancia)):
    if df2["B1"][i] == 1  and df2["B2"][i] == 1:
        resultados.append("Certo")
        
    if df2["B1"][i] == 1  and df2["B2"][i] == 2:
        resultados.append("Falso Certo")
        
    if df2["B1"][i] == 2  and df2["B2"][i] == 2:
        resultados.append("Errado")
        
    if df2["B1"][i] == 2  and df2["B2"][i] == 1:
        resultados.append("Falso Errado")
        
df2["B3"] = pd.Series(resultados) 


(df2["B3"].value_counts(normalize=True)*100).round(decimals=1).reindex(['Certo', 'Falso Certo','Errado','Falso Errado'])


Certo           27.0
Falso Certo      7.0
Errado          63.0
Falso Errado     3.0
Name: B3, dtype: float64

In [54]:
fFinal = []
for i in range(len(listaFinalRelevancia)):
    if df2["B3"][i] == "Certo"  or df2["B3"][i] == "Errado":
        fFinal.append("Sucesso")
    if df2["B3"][i] == "Falso Certo"  or df2["B3"][i] == "Falso Errado":
        fFinal.append("Falha")
    
        
df2["B4"] = pd.Series(fFinal)

(df2["B4"].value_counts(normalize=True)*100).round(decimals=1).reindex(['Sucesso', 'Falha'])

Sucesso    90.0
Falha      10.0
Name: B4, dtype: float64

In [55]:
writer = pd.ExcelWriter('fifa 17.xlsx')
df.to_excel(writer,'Treinamento')
df2.to_excel(writer,'Teste')
writer.save()   

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


   Fizemos um classificador para ensinar a máquina a classificar tweets relacionados ao nosso produto em relevantes ou não relevantes. O produto escolhido foi o Fifa 2017, um dos jogos virtuais mais populares no mundo que sempre gera uma grande quantidade de comentários sobre. Procuramos fazer um classificador que identificassem os comentários relevantes sobre o game, ou seja que fornecessem alguma forma de feedback sobre a experiência dos usuários.

   Com os tweets processados pelo classificador, obtivemos 27% de certos relevantes (o classificador e nós consideramos relevante), 7% de falsos relevantes (o classificador considera relevante mas nós não), 63% de certos irrelevantes (o classificador e nós consideramos irrelevantes) e 3% de falsos irrelevantes (o classificador considera irrelevante mas nós não). No total, o classificador acertou 90% da relevância dos tweets em relação a nossa definição de relevância e irrelevância.

   Mensagens com dupla negação e sarcasmo não interferem no processamento do classificador pois mesmo contendo alguma das duas, elas não mudam o fato do tweet ser considerado um feedback ou não. 
   
   Num futuro próximo, o classificador poderá ser otimizado com alguns passos já traçados. Como unir alguns verbos que estão conjugados de maneiras diferentes em uma só palavra, palavras que tem algumas variações também. Aumentar a base de tweets inicial para assim ter um banco de palavras maior e com as probabilidades de cada uma mais precisas. Isso seria muito vantajoso para detectar a satisfação do público e as correções que devem ser feitas para o futuro da série.